<a href="https://colab.research.google.com/github/trong-shen/Game-of-Throne-Project/blob/master/GOT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the CSV file from Github

In [7]:
import pandas as pd

GOT1= pd.read_csv('https://raw.githubusercontent.com/trong-shen/Game-of-Throne-Project/master/Game_of_Thrones_Script.csv')

GOT1.head()

,Release Date,Season,Episode,Episode Title,Name,Sentence
0,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,What do you expect? They're savages. One lot s...
1,2011-04-17,Season 1,Episode 1,Winter is Coming,will,I've never seen wildlings do a thing like this...
2,2011-04-17,Season 1,Episode 1,Winter is Coming,waymar royce,How close did you get?
3,2011-04-17,Season 1,Episode 1,Winter is Coming,will,Close as any man would.
4,2011-04-17,Season 1,Episode 1,Winter is Coming,gared,We should head back to the wall.


In [8]:
!pip install nltk
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
tokenized_word=[];
#Create a table with tokenized words
for i in range(0,len(GOT1)):
  tokenized_word.append(nltk.word_tokenize(GOT1.Sentence[i]))
GOT1.insert(6,'Tokenized_Sentence',tokenized_word)


In [0]:
GOT1.Tokenized_Sentence[0][0]

'What'

In [9]:
GOT1.tail()

,Release Date,Season,Episode,Episode Title,Name,Sentence
23906,2019-05-19,Season 8,Episode 6,The Iron Throne,brienne,I think we can all agree that ships take prece...
23907,2019-05-19,Season 8,Episode 6,The Iron Throne,bronn,I think that's a very presumptuous statement.
23908,2019-05-19,Season 8,Episode 6,The Iron Throne,tyrion lannister,I once brought a jackass and a honeycomb into ...
23909,2019-05-19,Season 8,Episode 6,The Iron Throne,man,The Queen in the North!
23910,2019-05-19,Season 8,Episode 6,The Iron Throne,all,The Queen in the North! The Queen in the North...
